In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [6]:
df = pd.read_csv('datasets/SatisfactionClients/cdiscount_retraite.csv', index_col=0)
df.sample(3)

,pays,note,titre,commentaire,Commentaire_filtre,titre_filtre
80794,FR,5,CLIENTE DEPUIS PLUS DE 10 ANNEE JE N'AI…,Il y a 14 heuresSur invitationCLIENTE DEPUIS P...,"['heuressur', 'invitationcliente', 'depuis', '...","['cliente', 'depuis', 'plus', 'annee']"
10510,FR,2,Frais de port à votre charge si vous…,ll y a 5 joursFrais de port à votre charge si ...,"['joursfrais', 'port', 'charge', 'frais', 'por...","['frais', 'port', 'charge']"
4615,FR,1,Très mauvaise expérience,ll y a 5 joursTrès mauvaise expérienceTrès mau...,"['jourstrès', 'mauvaise', 'expériencetrès', 'm...","['très', 'mauvaise', 'expérience']"


In [63]:
# séparation de la variable cible et des variables explicatives
X = df['Commentaire_filtre']
y = df['note']

In [64]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [65]:
# conversion des chaines de caratères en tokens numériques
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [112]:
# application du modèle de RandomForestClassifier sur les tokens numériques et calcul des prédictions
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=1, n_jobs=-1)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# calcul des métriques des résultats du modèle
print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['données prédites'])
display(cm)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14958
           2       1.00      1.00      1.00      2470
           4       1.00      1.00      1.00      1205
           5       1.00      1.00      1.00      6191

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


données prédites,1,2,4,5
données réelles,,,,
1,14958,0,0,0
2,0,2470,0,0
4,0,0,1205,0
5,0,0,0,6191


les prédictions sur le test remontent les notes suivantes :  [1 1 1 1]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 1 
 Bel article mais malheureusement trop petit  - la note est de : 1 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 1 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 1


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [111]:
clf2 = RandomForestClassifier(n_estimators=10, n_jobs=-1)

clf2.fit(X_train, y_train)

y_pred2 = clf2.predict(X_test)

print(clf2.score(X_test, y_test), end='\n\n')

cr2 = classification_report(y_test, y_pred2)
print(cr2)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm2 = pd.crosstab(y_test, y_pred2, rownames=['données réelles'], colnames=['données prédites'])
display(cm2)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf2.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14958
           2       1.00      1.00      1.00      2470
           4       1.00      1.00      1.00      1205
           5       1.00      1.00      1.00      6191

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


données prédites,1,2,4,5
données réelles,,,,
1,14958,0,0,0
2,0,2470,0,0
4,0,0,1205,0
5,0,0,0,6191


les prédictions sur le test remontent les notes suivantes :  [1 1 1 1]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 1 
 Bel article mais malheureusement trop petit  - la note est de : 1 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 1 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 1


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [110]:
clf3 = RandomForestClassifier(n_estimators=100, n_jobs=-1)

clf3.fit(X_train, y_train)

y_pred3 = clf3.predict(X_test)

print(clf3.score(X_test, y_test), end='\n\n')

cr3 = classification_report(y_test, y_pred3)
print(cr3)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm3 = pd.crosstab(y_test, y_pred3, rownames=['données réelles'], colnames=['données prédites'])
display(cm3)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf3.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14958
           2       1.00      1.00      1.00      2470
           4       1.00      1.00      1.00      1205
           5       1.00      1.00      1.00      6191

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



données prédites,1,2,4,5
données réelles,,,,
1,14958,0,0,0
2,0,2470,0,0
4,0,0,1205,0
5,0,0,0,6191


les prédictions sur le test remontent les notes suivantes :  [1 1 1 1]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 1 
 Bel article mais malheureusement trop petit  - la note est de : 1 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 1 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 1


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [109]:
clf4 = RandomForestClassifier()

clf4.fit(X_train, y_train)

y_pred4 = clf4.predict(X_test)

print(clf4.score(X_test, y_test), end='\n\n')

cr4 = classification_report(y_test, y_pred4)
print(cr4)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm4 = pd.crosstab(y_test, y_pred4, rownames=['données réelles'], colnames=['données prédites'])
display(cm4)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf4.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14958
           2       1.00      1.00      1.00      2470
           4       1.00      1.00      1.00      1205
           5       1.00      1.00      1.00      6191

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



données prédites,1,2,4,5
données réelles,,,,
1,14958,0,0,0
2,0,2470,0,0
4,0,0,1205,0
5,0,0,0,6191


les prédictions sur le test remontent les notes suivantes :  [1 1 5 1]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 1 
 Bel article mais malheureusement trop petit  - la note est de : 1 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 5 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 1


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


###### Le modèle RandomClassifier semble faire de l'over featting systématiquement 